### 0. 분석 환경 설정

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
from soyclustering import SphericalKMeans
from scipy.sparse import csr_matrix
from sklearn.feature_selection import f_regression
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVR
import warnings
import random

os.chdir(r"C:\Users\GilseungAhn\Desktop\한양대\3. 프로젝트\2. 진행중인 과제\포스트코로나 AI 챌린지")
warnings.filterwarnings("ignore")
np.random.seed(0)

### 1. 필요 함수 정의

#### make_wcluster_document_df: 단어군집 - 문서 행렬 생성 함수

In [2]:
# make_wcluster_document_df: 단어 - 문서 행렬을 단어 군집 - 문서 행렬로 변환하는 함수
def make_wcluster_document_df(_word_document_matrix, num_cluster, date_values): # _ (prefix)는 기존 데이터와 함수 입력이 충돌하지 않도록 넣은 것    
    # 일자 변수 저장
    #date_values = _word_document_matrix['게시날짜'].values  
    
    if '게시날짜' in _word_document_matrix.columns:
        _word_document_matrix.drop('게시날짜', axis = 1, inplace = True)
    
    #_word_document_matrix.drop('게시날짜', axis = 1, inplace = True)
    
    # transpose를 사용하여 단어 - 문서 행렬을 문서 - 단어 행렬로 변환
    document_word_matrix = _word_document_matrix.T
    
    # 군집화 수행 및 군집 결과 사전화 (key: 단어, value: 군집 번호)
    spherical_kmeans = SphericalKMeans(n_clusters = num_cluster, max_iter = 1000, verbose = 0, init = 'similar_cut')    
    word_cluster_label = spherical_kmeans.fit_predict(csr_matrix(document_word_matrix))
    word_cluster_dict = pd.Series(word_cluster_label, index = list(_word_document_matrix.columns)).apply(lambda x:"단어군집_" + str(x+1)).to_dict()    
    
    # 단어로 된 컬럼 이름을 군집 번호로 바꾸기
    _word_document_matrix.rename(columns = word_cluster_dict, inplace = True)
    
    # 각 군집에 속한 단어 등장 횟수 합계 계산 
    wcluster_document_df = pd.DataFrame()
    for x in range(1, num_cluster + 1):
        try:
            column_values = _word_document_matrix["단어군집_" + str(x)].sum(axis = 1).values
        except:
            column_values = _word_document_matrix["단어군집_" + str(x)]
        wcluster_document_df["단어군집_" + str(x)] = column_values
        
    # 기초 데이터와 병합을 위한 게시 날짜 변수 추가
    wcluster_document_df['게시날짜'] = date_values
    
    # 날짜별 단어_군집 변수 합계 계산
    cluster_columns = ["단어군집_" + str(x) for x in range(1, num_cluster + 1)]
    wcluster_document_df = wcluster_document_df.groupby('게시날짜', as_index = False)[cluster_columns].sum()
    
    # 각 군집에 속한 단어 등장 비율 계산 
    wcluster_document_df[cluster_columns] = wcluster_document_df[cluster_columns].values / wcluster_document_df[cluster_columns].sum(axis = 1).values.reshape(len(wcluster_document_df), 1)
        
    return word_cluster_dict, wcluster_document_df

#### 학습 데이터 생성:  _wcluster_document_df와 _base_df 병합

In [3]:
def generate_training_data(_wcluster_document_df, _base_df, K): 
    _wcluster_document_df['게시날짜'] = pd.to_datetime(_wcluster_document_df['게시날짜'])    
    for d in range(4):
        _base_df["D+" + str(d) + "_이전일자"] = pd.to_datetime(_base_df["D+" + str(d) + "_이전일자"])
        _base_df = pd.merge(_base_df, _wcluster_document_df, left_on = "D+" + str(d) + "_이전일자", right_on = '게시날짜').drop('게시날짜', axis = 1)
        for col in _wcluster_document_df.columns:
            _base_df.rename(columns = {col:"D+" + str(d) + "_이전일자_" + col}, inplace = True)

    _base_df.sort_values(by = '일자', inplace = True)
    day_list = _base_df['일자'].unique()

    # 일자 기준 데이터 분리 (2주까지 데이터가 예측)
    train_index = _base_df['일자'] < day_list[-14]
    test_index = _base_df['일자'] >= day_list[-14]

    train_df = _base_df.loc[train_index]
    test_df = _base_df.loc[test_index]
    
    # 최근 데이터 재샘플링 수행
    if K > 0:
        sample_weight = np.array(np.exp(K * np.arange(len(day_list)) / len(day_list)), dtype = int) - 1    
        sample_weight_dict = pd.Series(sample_weight, index = day_list).to_dict()
        for day, weight in sample_weight_dict.items():
            if weight > 0:
                duplicated_samples = train_df[train_df['일자'] == day]
                train_df = train_df.append([duplicated_samples] * weight, ignore_index = True)

    # 특징 벡터와 라벨 분리
    Train_X = train_df.drop(['신규확진자', '일자', 'D+0_이전일자', 'D+1_이전일자', 'D+2_이전일자', 'D+3_이전일자'], axis = 1)    
    Test_X = test_df.drop(['신규확진자', '일자', 'D+0_이전일자', 'D+1_이전일자', 'D+2_이전일자', 'D+3_이전일자'], axis = 1)    
    
    Train_Y = train_df['신규확진자']    
    Test_Y = test_df['신규확진자']
    
    return Train_X, Test_X, Train_Y, Test_Y

#### 특징 점수 기준 정렬

In [4]:
def feature_ranking(Train_X, Train_Y):    
    output = pd.Series(f_regression(Train_X, Train_Y)[1], index = Train_X.columns).sort_values()
    D_ind = output.index.tolist().index('D')
    return pd.Index(['D'] + output.index.tolist()[:D_ind] + output.index.tolist()[D_ind+1:])

#### 모델 목록 생성

In [6]:
def make_model_list():
    model_list = []
    model_parameter_dict = dict()
    
    model_parameter_dict[SVR] = ParameterGrid({"epsilon": [0.1, 0.5, 1, 2, 5, 10],
                                              "C": [2**-6, 2**-5, 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1, 2**2, 2**3],
                                              "kernel": ['linear', 'rbf', 'poly'],
                                              "gamma": [1e-08, 1e-07, 1e-06, 1e-05, 1e-04, 1e-03, 1e-02, 1e-01, 1, 10, 100, 'scale', 'auto'],
                                               "degree": [1, 2, 3, 4],
                                              "max_iter": [50, 100, 1000, 10000, 100000]})
                                                                  
    model_name_list = []
    model_parameter_list = []
    
    for model in model_parameter_dict.keys():
        for parameter in model_parameter_dict[model]:   
            if parameter['kernel'] == 'linear' and parameter['gamma'] != 1e-08:
                pass
            elif parameter['kernel'] != 'poly' and parameter['degree'] != 1:
                pass

            model_list.append(model(**parameter))
            model_name_list.append(str(model).split('(')[0].split('.')[-1][:-2])
            parameter_name = str(parameter).replace('"', '')
            parameter_name = parameter_name.replace("'", '')
            parameter_name = parameter_name.replace("{", '')
            parameter_name = parameter_name.replace("}", '')

            model_parameter_list.append(parameter_name)
    
    return model_list, model_name_list, model_parameter_list

#### 모델 평가 (제공)

In [7]:
def model_test(_model, _Text_X, _Test_Y):
    pred_Y = _model.predict(_Text_X)
    pred_Y[pred_Y < 0] = 0
    pred_Y = np.round(pred_Y)
    Num = sum((_Test_Y - pred_Y) ** 2)
    Dem = sum(_Test_Y ** 2)
    score = 100 * (1- Num / Dem)
    return score, np.std(pred_Y)

### 2. 튜닝 수행

#### 2.1 튜닝 범위 설정

In [8]:
n_clusters_list = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 100]
n_feature_list = np.arange(50, 501, 50)
K_list = [0, 1, 2, 3, 4, 5]
model_list, model_name_list, model_parameter_list = make_model_list()
max_iteration = 100000000

#### 2.2 튜닝 수행

In [9]:
word_document_matrix = pd.read_csv("데이터/3. 정제데이터/단어-문서행렬.csv", encoding = "utf8")    
date_values = word_document_matrix['게시날짜'].values
base_df = pd.read_csv("데이터/3. 정제데이터/기초데이터.csv", encoding = "utf8")

# 튜닝 결과 저장 파일 및 헤더 생성
f = open("분석 결과 및 모델/파라미터튜닝결과.txt", "w")
f.write("num_cluster\tK\tk\tC\tdegree\tepsilon\tgamma\tkernel\tmax_iter\tscore\tscore_std\n")
f.close()

i = 0
best_score = 0

while i < max_iteration:
    word_document_matrix = pd.read_csv("데이터/3. 정제데이터/단어-문서행렬.csv", encoding = "utf8")    
    date_values = word_document_matrix['게시날짜'].values
    base_df = pd.read_csv("데이터/3. 정제데이터/기초데이터.csv", encoding = "utf8")
    
    num_cluster = random.choice(n_clusters_list)
    K = random.choice(K_list)
    k = random.choice(n_feature_list)
    _, wcluster_document_df = make_wcluster_document_df(word_document_matrix, num_cluster, date_values)
    Train_X, Test_X, Train_Y, Test_Y = generate_training_data(wcluster_document_df, base_df, K)
    feature_rank = feature_ranking(Train_X, Train_Y)        
    selected_features = list(set(list(feature_rank[:k]) + ['D', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']))
    feature_selected_Train_X = Train_X[selected_features]
    feature_selected_Test_X = Test_X[selected_features]            

    for (model, model_name, parameter_name) in zip(model_list, model_name_list, model_parameter_list):
        model.fit(feature_selected_Train_X, Train_Y)
        score, score_std = model_test(model, feature_selected_Test_X, Test_Y)   
        if score > best_score:
            best_score = score
            best_parameter = parameter_name
            best_K = K
            f = open("분석 결과 및 모델/파라미터튜닝결과.txt", "a")
            f.write(str(num_cluster))
            f.write('\t')
            f.write(str(K))
            f.write('\t')
            f.write(str(k))
            f.write('\t')
            f.write(best_parameter)
            f.write('\t')
            f.write(str(score))
            f.write('\t')
            f.write(str(score_std))
            f.write('\n')
            f.close()
            print("이터레이션: {}/{}, 최고점수: {}, 표준편차: {}".format(i, max_iteration, best_score, score_std))

        i += 1
                

이터레이션: 0/16848000, 최고점수: 81.27460629921259, 표준편차: 1.3941301822131522
이터레이션: 195/16848000, 최고점수: 81.70521653543307, 표준편차: 1.3251984930593432
이터레이션: 230/16848000, 최고점수: 82.19734251968505, 표준편차: 0.4729422800359504
이터레이션: 245/16848000, 최고점수: 83.50147637795276, 표준편차: 0.3460225292082949
이터레이션: 250/16848000, 최고점수: 84.00590551181102, 표준편차: 0.0
이터레이션: 585/16848000, 최고점수: 85.00246062992126, 표준편차: 0.0
이터레이션: 37940/16848000, 최고점수: 85.72834645669292, 표준편차: 0.33165520597480563
